# MNIST-minus-minus: Punky data generation

A handwritten-digit reading task, now with way, way more chaos!

## Authors
- **David W Hogg** (NYU) (Flatiron)
- **Soledad Villar** (JHU)

## To-Do / Bugs:
- Show that MNIST digits are distinguished not by READING the digits.

## Notes
- Some content copied from <https://github.com/wxs/keras-mnist-tutorial/blob/master/MNIST%20in%20Keras.ipynb>.
- Some content copied from <https://colab.research.google.com/github/tensorflow/tpu/blob/master/tools/colab/fashion_mnist.ipynb>.

In [ ]:
import numpy as np
import pickle
import os
from keras.datasets import mnist
rng = np.random.default_rng(17) # The most random of all possible seeds

In [ ]:
n_train = 60_000 # number of training-set examples to make for each data set
n_test  = 10_000 # number of test-set     examples to make for each data set

In [ ]:
# the original MNIST data, split between train and test sets
(X0_train, y0_train), (X0_test, y0_test) = mnist.load_data()
M0_train = np.zeros((len(X0_train), 2, 2)).astype("int32")
M0_train[:, 0, 0] = 1.
M0_train[:, 1, 1] = 1.
M0_test = np.zeros((len(X0_test), 2, 2)).astype("int32")
M0_test[:, 0, 0] = 1.
M0_test[:, 1, 1] = 1.
print("X0_train shape", X0_train.shape)
print("M0_train shape", M0_train.shape)
print("y0_train shape", y0_train.shape)
print("X0_test shape", X0_test.shape)
print("M0_test shape", M0_test.shape)
print("y0_test shape", y0_test.shape)

In [ ]:
def transform(Xs, rng):
    (n, n1, n2) = Xs.shape
    Xm = Xs.copy()
    Mm = np.zeros((n, 2, 2)).astype("int32")
    R1 = np.array([[0, 1], [-1, 0]])
    for i,X in enumerate(Xs):
        M = np.eye(2)
        X1 = X.copy()
        if rng.random() < 0.5:
            X1 = np.flip(X1, axis=1)
            M[1] *= -1 # HACK; GUESS
        nrot = rng.integers(0,4)
        for j in range(nrot):
            X1 = np.rot90(X1, 1)
            M = M @ R1
        Xm[i] = X1
        Mm[i] = M
    return Xm, Mm

def minusify_1(X, y, N, rng, noisify=False, jitter=False):
    (n, n1, n2) = X.shape
    assert n1 == n2
    assert y.shape == (n, )
    inds = rng.integers(0, n, N)
    ym = np.zeros(N).astype("uint8")
    ym = y[inds]
    Xm, Mm = transform(X[inds], rng)
    if noisify:
        Xm += rng.integers(5, size=Xm.shape).astype("uint8")
    if jitter:
        for i in range(N):
            Xm[i] = np.roll(Xm[i], rng.integers(-2, 3), axis=0)
            Xm[i] = np.roll(Xm[i], rng.integers(-2, 3), axis=1)
    return Xm, ym, Mm

def minusify_4(X, y, N, rng):
    (n, n1, n2) = X.shape
    assert n1 == n2
    assert y.shape == (n, )
    inds = [rng.integers(0, n, N) for i in range(4)]
    ym = np.zeros(N).astype(int)
    for i in range(4):
        ym = ym + 10**i * y[inds[i]].astype(int)
    ym = np.array(["{:04d}".format(y) for y in ym])
    Xmm = np.concatenate((np.concatenate((X[inds[3]], X[inds[2]]), axis=2),
                          np.concatenate((X[inds[1]], X[inds[0]]), axis=2)),
                         axis=1)
    Xm, Mm = transform(Xmm, rng)
    return Xm, ym, Mm

def minusify_9(X, y, N, rng):
    foo = 3
    (n, n1, n2) = X.shape
    assert n1 == n2
    assert y.shape == (n, )
    inds = rng.integers(0, n, size=(N, foo, foo))
    ym = y[inds[:, 1, 1]].astype("uint8")
    Xmm = np.concatenate([np.concatenate([X[inds[:, i, j]] for i in range(foo)],
                                         axis=2) for j in range(foo)], axis=1)
    Xm, Mm = transform(Xmm, rng)
    return Xm, ym, Mm

In [ ]:
def infinity_transform(Xs, n1, n2, rng):
    (n, in_n1, in_n2) = Xs.shape
    Xm = np.zeros((n, n1, n2))
    xyout = np.array(np.meshgrid(np.arange(0.25, n1, 0.5) - 0.5 * n1, np.arange(0.25, n2, 0.5) - 0.5 * n2))
    Mm = np.zeros((n, 2, 2)).astype("float32")
    for i,X in enumerate(Xs):
        eigratio = 0.0
        while eigratio < 0.333:
            M = rng.normal(size=(2, 2))
            u, s, v = np.linalg.svd(M)
            eigratio = s[1] / s[0]
        M /= np.sqrt(np.prod(s))
        xyin = np.tensordot(M, xyout, (1, 0))
        yin = np.floor(np.clip(xyin[0] + 0.5 * in_n1, 0., in_n1-0.5)).astype(int) ## HACKS
        xin = np.floor(np.clip(xyin[1] + 0.5 * in_n2, 0., in_n2-0.5)).astype(int)
        foo = X[yin, xin]
        X1 = foo.reshape(n1, 2, n2, 2).mean(-1).mean(1) # bin down 2x2
        Xm[i] = X1
        Mm[i] = np.flip(np.linalg.inv(M).T, axis=1) # I don't understand why this HACK is necessary
    return Xm, Mm

def minusify_infinity(X, y, N, rng):
    foo = 7
    (n, n1, n2) = X.shape
    assert n1 == n2
    assert y.shape == (n, )
    inds = rng.integers(0, n, size=(N, foo, foo))
    ym = y[inds[:, foo // 2, foo // 2]].astype("uint8")
    Xmm = np.concatenate([np.concatenate([X[inds[:, i, j]] for i in range(foo)],
                                         axis=2) for j in range(foo)], axis=1)
    Xm, Mm = infinity_transform(Xmm, 3 * n1, 3 * n2, rng)
    return (Xm + 0.5).astype("uint8"), ym, Mm # Note +0.5 rounding HACK!

In [ ]:
def pickle_dataset(filename, Xtrain, Mtrain, ytrain, Xtest, Mtest, ytest):
    with open(filename, "wb") as file:
        pickle.dump(((Xtrain, Mtrain, ytrain), (Xtest, Mtest, ytest)), file)
    os.system("gzip --force --best " + filename + " &")

In [ ]:
I69 = np.array([y in (6, 9) for y in y0_train])
X_train69, y_train69, M_train69 = minusify_1(X0_train[I69], y0_train[I69], n_train, rng, jitter=True)
I69 = np.array([y in (6, 9) for y in y0_test])
X_test69, y_test69, M_test69 = minusify_1(X0_test[I69], y0_test[I69], n_test, rng, jitter=True)

In [ ]:
pickle_dataset("SixtyNine++.pkl", X_train69, M_train69, y_train69, X_test69, M_test69, y_test69)

In [ ]:
npix = 4
X_trainLow = X0_train.reshape(-1, npix, 28 // npix, npix, 28 // npix).mean(-1).mean(2) # bin down
X_testLow = X0_test.reshape(-1, npix, 28 // npix, npix, 28 // npix).mean(-1).mean(2) # bin down

In [ ]:
X_trainLow.shape

In [ ]:
pickle_dataset("LowRes++.pkl", X_trainLow, M0_train, y0_train, X_testLow, M0_test, y0_test)

In [ ]:
marg = 7
X_trainCut = 1 * X0_train
X_trainCut[:, marg:-marg, marg:-marg] = 0
X_testCut = 1 * X0_test
X_testCut[:, marg:-marg, marg:-marg] = 0

In [ ]:
pickle_dataset("CutOut++.pkl", X_trainCut, M0_train, y0_train, X_testCut, M0_test, y0_test)

In [ ]:
X_trainProj = 0 * X0_train
X_trainProj[:, :, 0] = np.mean(X0_train, axis=2)
X_trainProj[:, 0, :] = np.mean(X0_train, axis=1)
X_testProj = 0 * X0_test
X_testProj[:, :, 0] = np.mean(X0_test, axis=2)
X_testProj[:, 0, :] = np.mean(X0_test, axis=1)

In [ ]:
pickle_dataset("Projections++.pkl", X_trainProj, M0_train, y0_train, X_testProj, M0_test, y0_test)

In [ ]:
def cropify(Xs):
    m = 18
    newXs = np.zeros((len(Xs), m, m)).astype("uint8")
    for i,X in enumerate(Xs):
        d1 = rng.integers(1, 28-m-1)
        d2 = rng.integers(1, 28-m-1)
        newXs[i] = X[d1:d1+m, d2:d2+m]
    return newXs

X_trainCrop = cropify(X0_train)
X_testCrop = cropify(X0_test)

In [ ]:
pickle_dataset("Crops++.pkl", X_trainCrop, M0_train, y0_train, X_testCrop, M0_test, y0_test)

In [ ]:
def jitter(Xs):
    newXs = np.zeros(Xs.shape).astype("uint8")
    for i,X in enumerate(Xs):
        d1 = rng.choice([0, 2])
        d2 = rng.choice([0, 2])
        newXs[i, 1:-1, 1:-1] = X[d1:d1+26, d2:d2+26]
    return newXs

# jitter ONLY the test set!
X_trainJit = 1 * X0_train
X_testJit = jitter(X0_test)

In [ ]:
pickle_dataset("Jitter++.pkl", X_trainJit, M0_train, y0_train, X_testJit, M0_test, y0_test)